In [1]:
from transformers import SiglipForImageClassification, AutoImageProcessor
import os
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

In [17]:
model = SiglipForImageClassification.from_pretrained("prithivMLmods/Human-Action-Recognition")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Human-Action-Recognition")

In [21]:
df = pd.read_csv("frames_annotations.csv")
df["label"] = df["label"].map({"not_taking_medication": 0, "taking_medication": 1})
dataset = Dataset.from_pandas(df)

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs["labels"] = example["label"]
    return {
        "pixel_values": inputs["pixel_values"].squeeze(),
        "labels": inputs["labels"]
    }

dataset = dataset.map(preprocess, remove_columns=["image", "label"])

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

In [22]:
dataset = dataset.train_test_split(test_size=0.1)

In [23]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [25]:
model.config.label2id = {"not_taking_medication": 0, "taking_medication": 1}
model.config.id2label = {0: "not_taking_medication", 1: "taking_medication"}

training_args = TrainingArguments(
    output_dir="./HAR-medication-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    hub_model_id="Adekiii/HAR-medication-finetuned",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.735300,0.624664,0.870588,0.000000,0.000000,0.000000
2,0.455000,0.388687,0.870588,0.000000,0.000000,0.000000
3,0.512700,0.422446,0.870588,0.000000,0.000000,0.000000


c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

TrainOutput(global_step=288, training_loss=0.5537669054336019, metrics={'train_runtime': 1014.3531, 'train_samples_per_second': 2.263, 'train_steps_per_second': 0.284, 'total_flos': 1.9223958439719936e+17, 'train_loss': 0.5537669054336019, 'epoch': 3.0})

In [27]:
trainer.save_model("./HAR-med-finetuned")
processor.save_pretrained("./HAR-med-finetuned")

['./HAR-med-finetuned\\preprocessor_config.json']

In [ ]:
from huggingface_hub import HfApi

access_token = "token_here"

api = HfApi(token=access_token)
api.upload_folder(
    folder_path="HAR-med-finetuned",
    repo_id="Adekiii/HAR-medication-finetuned", # change with own repo
    repo_type="model",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adekiii/HAR-medication-finetuned/commit/358876b93413718f0f46962ca617d85be23fdf93', commit_message='Upload folder using huggingface_hub', commit_description='', oid='358876b93413718f0f46962ca617d85be23fdf93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Adekiii/HAR-medication-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Adekiii/HAR-medication-finetuned'), pr_revision=None, pr_num=None)